In [45]:
import sqlite3
import math
import os
import time
import pandas as pd
import numpy as np
import ipywidgets as widgets
import seaborn as sns
import folium
from folium import plugins
from selenium import webdriver
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output

In [44]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/FPA_FOD_20170508.sqlite")
df = pd.read_sql_query("SELECT * from Fires", con)

In [3]:
#Adding values to a map https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/
#Interactive wildfire map https://towardsdatascience.com/creating-an-interactive-map-of-wildfire-data-using-folium-in-python-7d6373b6334a

In [46]:
#Not used currently, but could be used to create a map of the US with state boarders.
data_url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
us_states_url = f"{data_url}/us-states.json"

"https://github.com/deldersveld/topojson/blob/master/countries/sweden/"

sweden-municipalities.json


In [ ]:

data_url = (
    "https://github.com/deldersveld/topojson/blob/master/countries/sweden/"
)
swedish_municipalities = f"{data_url}/sweden-municipalities.json"

In [ ]:
#map2 = folium.Map(location=[43, -100], tiles='Stamen Terrain', zoom_start=4)

#marker_cluster = folium.MarkerCluster().add_to(map2)

#for point in range(0, len(firesArr)):
   #folium.Marker(firesArr[point], popup=df['Name'][point], icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')).add_to(marker_cluster)
#map2

In [48]:
#A copy of the dataframe that only includes 8 of the columns
df_copy = df[['FIRE_NAME', 'FIRE_YEAR', 
         'SOURCE_REPORTING_UNIT_NAME', 'STAT_CAUSE_DESCR',
         'FIRE_SIZE', 'LATITUDE',
         'LONGITUDE', 'STATE']]

In [49]:
#drop all fires smaller than 1000 acres and show how many fires are left
df_copy = df_copy.where(df['FIRE_SIZE'] > 1000).dropna()
x = len(df)
print(df_copy.shape)

print ('The new dataframe is',
       round(len(df_copy)/x * 100, 3), 
       '% of the original dataset')

(10350, 8)
The new dataframe is 0.55 % of the original dataset


In [50]:
#Group fires by size
small_wildfires = folium.FeatureGroup(name = '< 10000 Acres')
medium_wildfires = folium.FeatureGroup(name = '10,000 - 50,000 Acres')
large_wildfires = folium.FeatureGroup(name = '50,000 - 100,000 Acres')
xl_wildfires = folium.FeatureGroup(name = '> 100,000 Acres')

In [51]:
#Function that takes the dataframe and returns a map. Circles on the map are colored based on the size of the fire.
def add_FireCircle(df, m):
    for i, v in df.iterrows():
        
        fire_size = float(v['FIRE_SIZE'])
        
        #When hoovering over the circle, the popup will show the fire name, year, cause, state and size.
        #More values can be added.
        popup = """
        Fire Name : <b>%s</b><br>
        Size (Acres) : <b>%s</b><br>
        State : <b>%s</b><br>
        Cause : <b>%s</b><br>
        Year: <b>%s</b><br>
        """ % (v['FIRE_NAME'], v['FIRE_SIZE'], 
            v['STATE'], v['STAT_CAUSE_DESCR'], 
            v['FIRE_YEAR'])
        
        
        if fire_size < 10000:
            folium.CircleMarker(location = [v['LATITUDE'], 
                                            v['LONGITUDE']],
                            radius = np.log(fire_size) * 0.8,
                            weight = 0,
                            tooltip = popup,
                            color = '#ffeda0',
                            fill_color = '#ffeda0',
                            fill_opacity = 0.7,
                            fill = True).add_to(small_wildfires)
            
        if fire_size in range(10000, 50000):
            folium.CircleMarker(location = [v['LATITUDE'], 
                                            v['LONGITUDE']],
                            radius = np.log(fire_size),
                            weight = 0,
                            tooltip = popup,
                            color = '#feb24c',
                            fill_color = '#feb24c',
                            fill_opacity = 0.7,
                            fill = True).add_to(medium_wildfires)
        
        if fire_size in range(50000, 100000):
            folium.CircleMarker(location = [v['LATITUDE'], 
                                            v['LONGITUDE']],
                            radius = np.log(fire_size) * 1.5,
                            weight = 0,
                            tooltip = popup,
                            color = '#fc4e2a',
                            fill_color = '#fc4e2a',
                            fill_opacity = 0.7,
                            fill = True).add_to(large_wildfires)

        if fire_size > 100000:
            folium.CircleMarker(location = [v['LATITUDE'], 
                                            v['LONGITUDE']],
                            radius = np.log(fire_size) * 2,
                            weight = 0,
                            tooltip = popup,
                            color = '##b10026',
                            fill_color = '#b10026',
                            fill_opacity = 0.7,
                            fill = True).add_to(xl_wildfires)

    small_wildfires.add_to(m)
    medium_wildfires.add_to(m)
    large_wildfires.add_to(m)
    xl_wildfires.add_to(m)
    folium.LayerControl(collapsed = False).add_to(m)

    return m


In [52]:
#Map instanciation, with start point at the center of the US. Type of map is set to Stamen Terrain.
map = folium.Map(location = [40.44, -104.81],
               tiles = 'Stamen Terrain',
               zoom_start = 6)

In [53]:
#Creating a dataframe with fires in Alaska.
alaska = df_copy.where(df_copy['STATE'] == 'AK').dropna()

In [54]:
#Creation of map for fires in Alaska and exporting it to a html file.
m_alaska = add_FireCircle(alaska, map)
m_alaska.save('alaska.html')